# Differential Drive FK — ICC Fix Version

This is a patched replacement notebook fixing the Step 4 ICC string bug.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

def vw_from_wheels(omega_L, omega_R, r, L):
    vL = r * omega_L
    vR = r * omega_R
    v = 0.5 * (vR + vL)
    w = (vR - vL) / L
    return v, w

def draw_robot(ax, x, y, th, L=0.3):
    fwd = np.array([np.cos(th), np.sin(th)])
    left = np.array([-np.sin(th), np.cos(th)])
    p1 = np.array([x, y]) + 0.18*L*fwd
    p2 = np.array([x, y]) - 0.12*L*fwd + 0.10*L*left
    p3 = np.array([x, y]) - 0.12*L*fwd - 0.10*L*left
    ax.fill([p1[0], p2[0], p3[0]], [p1[1], p2[1], p3[1]], alpha=0.4)

x0 = widgets.FloatSlider(value=0.0, min=-2, max=2, step=0.05, description="x0")
y0 = widgets.FloatSlider(value=0.0, min=-2, max=2, step=0.05, description="y0")
th0 = widgets.FloatSlider(value=0.0, min=-180, max=180, step=1, description="θ0 (deg)")
omegaL = widgets.FloatSlider(value=2.0, min=-10, max=10, step=0.1, description="ω_L")
omegaR = widgets.FloatSlider(value=4.0, min=-10, max=10, step=0.1, description="ω_R")
r = widgets.FloatSlider(value=0.05, min=0.01, max=0.20, step=0.005, description="r")
L = widgets.FloatSlider(value=0.30, min=0.10, max=0.80, step=0.01, description="L")

out = widgets.Output()

def icc_demo(x0, y0, th0_deg, omega_L, omega_R, r, L):
    th = np.deg2rad(th0_deg)
    v, w = vw_from_wheels(omega_L, omega_R, r, L)

    fig, ax = plt.subplots(figsize=(6.5, 6.5))
    ax.grid(True, alpha=0.3)
    ax.set_aspect("equal", adjustable="box")
    ax.set_xlim(-3, 3)
    ax.set_ylim(-3, 3)

    ax.scatter([x0], [y0], s=60)
    draw_robot(ax, x0, y0, th, L=L)

    if abs(w) > 1e-8:
        R = v / w
        icc = np.array([x0, y0]) + R * np.array([-np.sin(th), np.cos(th)])
        ax.scatter([icc[0]], [icc[1]], s=90, marker="x")

        ang = np.linspace(0, 2*np.pi, 250)
        ax.plot(icc[0] + abs(R)*np.cos(ang),
                icc[1] + abs(R)*np.sin(ang),
                linestyle="--", alpha=0.6)

        # FIXED STRING (escaped newline)
        ax.text(-2.8, 2.3,
                f"v={v:.3f} m/s, ω={w:.3f} rad/s\nR=v/ω={R:.3f} m",
                bbox=dict(boxstyle="round", alpha=0.15))

    plt.show()

ui = widgets.VBox([
    widgets.HBox([x0, y0, th0]),
    widgets.HBox([omegaL, omegaR]),
    widgets.HBox([r, L])
])

widgets.interactive_output(icc_demo, {
    "x0": x0, "y0": y0, "th0_deg": th0,
    "omega_L": omegaL, "omega_R": omegaR,
    "r": r, "L": L
})

display(ui)
icc_demo(x0.value, y0.value, th0.value, omegaL.value, omegaR.value, r.value, L.value)
